In [25]:
import numpy as np
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
import random
from collections import defaultdict
from openpyxl.styles import PatternFill

NUM_ROOMS = 10
NUM_LABS = 4
NUM_PROFESSORS = 10
NUM_SECTIONS = 10
NUM_COURSES = 30
NUM_THEORY_COURSES = 25
NUM_LAB_COURSES = 5
TIMESLOTS_PER_DAY = 6
DAYS_PER_WEEK = 5
MORNING_SESSION_START = 0
AFTERNOON_SESSION_START = 3
POPULATION_SIZE = 100
MAX_GENERATIONS = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1
NUM_SEMESTERS = 5
CHROMOSOME_LENGTH = 13

course_names = {
    1: "PF", 2: "AP", 3: "IICT", 4: "Calculus", 5: "English", 6: "OOP", 7: "DLD", 8: "CPS",
    9: "Pak Studies", 10: "DE", 11: "Data St", 12: "Discrete St", 13: "COAL", 14: "LA", 15: "German",
    16: "DB", 17: "OS", 18: "Algo", 19: "Probability", 20: "Psychology", 21: "CNET", 22: "SDA",
    23: "Automata", 24: "Statistical Modelling", 25: "TBW", 26: "LAB-PF", 27: "LAB-OOP",
    28: "LAB-Data St", 29: "LAB-DB", 30: "LAB-CNET"
}

def generate_sections():
    sections = {}
    for i in range(NUM_SECTIONS):
        if i < 2: sections[chr(65 + i)] = (1, random.randint(30, 110))
        elif i < 4: sections[chr(65 + i)] = (2, random.randint(30, 110))
        elif i < 6: sections[chr(65 + i)] = (3, random.randint(30, 110))
        elif i < 8: sections[chr(65 + i)] = (4, random.randint(30, 110))
        elif i < 10: sections[chr(65 + i)] = (5, random.randint(30, 110))
    return sections

def create_timeslots():
    timeslots = []
    start_times = ["8:30", "10:00", "11:30", "1:00", "2:30", "4:00"]
    end_times = ["9:50", "11:20", "12:50", "2:20", "3:50", "5:20"]
    for start_time, end_time in zip(start_times, end_times):
        timeslot_str = f"{start_time}-{end_time}"
        timeslots.append(timeslot_str)
    return timeslots



def get_course_color(course_id):
    if 1 <= course_id <= 5 or course_id == 26: return "FFFF00"
    elif 6 <= course_id <= 10 or course_id == 27: return "FFA500"
    elif 11 <= course_id <= 15 or course_id == 28: return "FFC0CB"
    elif 16 <= course_id <= 20 or course_id == 29: return "ADD8E6"
    elif 21 <= course_id <= 25 or course_id == 30: return "90EE90"
    else: return "FFFFFF"

def calculate_fitness(chromosome, rooms, courses, sections, professors):
    fitness = 0
    used_slots = defaultdict(list)
    used_professors = defaultdict(list)
    used_rooms = defaultdict(list)

    for gene in chromosome:
        course_id, course_type, section, section_strength, professor, day1, timeslot1, room1 = gene

        # Constraint 1: Classes can only be scheduled in free classrooms.
        if (day1, timeslot1, room1) in used_rooms:
            fitness += 1

        # Constraint 2: A classroom should be big enough to accommodate the section.
        if sections[section][0] > rooms[room1][1]:
            fitness += 1

        # Constraint 3: A professor should not be assigned two different lectures at the same time.
        if (day1, timeslot1) in used_professors[professor]:
            fitness += 1

        # Constraint 4: The same section cannot be assigned to two different rooms at the same time.
        if (day1, timeslot1) in used_slots[section]:
            fitness += 1

        # Constraint 5: A room cannot be assigned for two different sections at the same time.
        if (day1, timeslot1, room1) in used_rooms:
            fitness += 1

        # Constraint 6: No professor can teach more than 3 courses.
        if len(used_professors[professor]) >= 3:
            fitness += 1

        # Constraint 7: No section can have more than 5 courses in a semester.
        if len(used_slots[section]) >= 5:
            fitness += 1

        # Constraint 8: Each course would have two lectures per week not on the same or adjacent days.
        for day, slot, room in used_rooms:
            if (day == day1 and abs(slot - timeslot1) == 1 and room == room1):
                fitness += 1

        used_slots[section].append((day1, timeslot1))
        used_professors[professor].append((day1, timeslot1))
        used_rooms[(day1, timeslot1, room1)].append(section)

    return fitness

def genetic_algorithm(rooms, courses, sections, professors):
    population = generate_initial_population(rooms, courses, sections, professors)
    best_chromosome = min(population, key=lambda c: calculate_fitness(c, rooms, courses, sections, professors))
    best_fitness = calculate_fitness(best_chromosome, rooms, courses, sections, professors)

    for generation in range(MAX_GENERATIONS):
        population = generate_new_population(population, rooms, courses, sections, professors)
        new_best_chromosome = min(population, key=lambda c: calculate_fitness(c, rooms, courses, sections, professors))
        new_best_fitness = calculate_fitness(new_best_chromosome, rooms, courses, sections, professors)

        if new_best_fitness < best_fitness:
            best_chromosome = new_best_chromosome
            best_fitness = new_best_fitness
            

    return best_chromosome

def generate_initial_population(rooms, courses, sections, professors):
    population = []
    professors_courses_count = defaultdict(int)
    
    for _ in range(POPULATION_SIZE):
        chromosome = []
        section_lectures_count = defaultdict(int) 
        section_rooms = defaultdict(lambda: defaultdict(list))  
        section_slots = defaultdict(list)  

        # Sequentially assign professors
        professor_index = 0

        # assigning for all sections
        for section, (semester, section_strength) in sections.items():
            
            # course assigning depending upon semester
            course_range_start = (semester - 1) * 5 + 1
            course_range_end = semester * 5 + 1
            lab_course = semester + 25

            assigned_theory_courses = set()
            assigned_lab_slots = set()
            section_room = None  

            # Sequential assignment of professors
            for course_id in range(course_range_start, course_range_end):
                theory_course_type = "Theory"
                
                # Professor Constraint
                professor = professors[professor_index % len(professors)]
                professor_index += 1

                professors_courses_count[course_id] += 1

                #selecting day and timeslot
                day1 = random.randint(0, DAYS_PER_WEEK - 1) 
                timeslot1 = random.randint(MORNING_SESSION_START, AFTERNOON_SESSION_START - 1)

                if section_room is None:
                    
                    section_room = random.choice([room for room, size in rooms if size >= section_strength])  #constraint 2
                    section_rooms[section][day1].append(section_room) 

                # Avoid scheduling lectures at the same time for the same section.
                while (day1, timeslot1) in section_slots[section]:
                    day1 = random.randint(0, DAYS_PER_WEEK - 1)
                    timeslot1 = random.randint(MORNING_SESSION_START, AFTERNOON_SESSION_START - 1)

                chromosome.append((course_id, theory_course_type, section, section_strength, professor, day1, timeslot1, section_room))
                section_slots[section].append((day1, timeslot1))  # Record slot usage.

                # Calculate next available slot for the same course on a different day.
                if day1 == 0: next_day = 2
                elif day1 == 1: next_day = 3
                elif day1 == 2: next_day = 0
                elif day1 == 3: next_day = 1
                elif day1 == 4: next_day = 1

                next_timeslot = timeslot1
                while (next_day, next_timeslot) in section_slots[section]:
                    next_timeslot = (next_timeslot + 1) % AFTERNOON_SESSION_START
                    if next_timeslot == 0: next_day = (next_day + 1) % DAYS_PER_WEEK

                # Assign the next lecture for the course.
                chromosome.append((course_id, theory_course_type, section, section_strength, professor, next_day, next_timeslot, section_room))
                section_slots[section].append((next_day, next_timeslot))  # Record slot usage.

            lab_professor = professors[professor_index % len(professors)]
            professor_index += 1

            day2 = random.randint(0, DAYS_PER_WEEK - 1)
            timeslot2 = random.randint(3, 4)

            lab_room2 = section_room
            assigned_lab_slots.add(timeslot2)
            chromosome.append((lab_course, "Lab", section, section_strength, lab_professor, day2, timeslot2, lab_room2))  
            assigned_lab_slots.add(timeslot2 + 1)
            chromosome.append((lab_course, "Lab", section, section_strength, lab_professor, day2, timeslot2 + 1, lab_room2))  

            section_lectures_count[section] += 6  

        if all(count == 6 for count in section_lectures_count.values()):
            population.append(chromosome)  

    return population



def generate_new_population(population, rooms, courses, sections, professors):
    new_population = []
    fitness_scores = [calculate_fitness(chromosome, rooms, courses, sections, professors) for chromosome in population]
    sorted_population = [chromosome for _, chromosome in sorted(zip(fitness_scores, population))]

    elitism_count = int(POPULATION_SIZE * 0.1)
    new_population.extend(sorted_population[:elitism_count])

    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = random.sample(sorted_population, 2)
        child1, child2 = crossover(parent1, parent2)
        mutate(child1, MUTATION_RATE)
        mutate(child2, MUTATION_RATE)
        new_population.append(child1)
        new_population.append(child2)

    return new_population[:POPULATION_SIZE]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, CHROMOSOME_LENGTH - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            gene = list(chromosome[i])
            mutation_point = random.randint(4, CHROMOSOME_LENGTH - 1)
            if mutation_point >= 4 and mutation_point < len(gene):
                if gene[1] == "Lab":
                    current_timeslot = gene[6]
                    if current_timeslot == 6:
                        new_timeslot = random.choice([3, 4])
                    else:
                        new_timeslot = current_timeslot 
                    gene[6] = new_timeslot

            chromosome[i] = tuple(gene)
    return chromosome



def write_to_excel(best_chromosome, rooms, timeslots):
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

    for day_idx, day_name in enumerate(day_names):
        ws = wb.create_sheet(day_name)
        header_row = [day_name] + timeslots
        ws.append(header_row)

        for room_idx, (room_id, room_capacity) in enumerate(rooms):
            room_semester = room_id % NUM_SEMESTERS
            room_number = f"C-{room_idx+301}"
            ws.cell(row=room_idx + 2, column=1, value=room_number)
            row = []
            for timeslot_idx, timeslot_str in enumerate(timeslots):
                course_info = find_course_by_timeslot(best_chromosome, day_idx, timeslot_idx, room_idx)
                if course_info:
                    course, professor, section, section_size = course_info
                    course_name = course_names[course]
                    cell_value = f"{course_name} ({section})|{professor}|Strength:{section_size}"

                    course_color = get_course_color(course)
                    cell = ws.cell(row=room_idx + 2, column=timeslot_idx + 2)
                    cell.fill = PatternFill(start_color=course_color, end_color=course_color, fill_type="solid")
                    cell.value = cell_value
                else:
                    row.append("")
    
    for idx, gene in enumerate(best_chromosome):
        print(f"Gene {idx+1}: {gene}")

                    

    wb.save("timetable3.xlsx")

def find_course_by_timeslot(chromosome, day_idx, timeslot_idx, room_idx):
    for gene in chromosome:
        course_id, course_type, section, section_strength, professor, day1, timeslot1, room1 = gene
        if day_idx == day1 and timeslot_idx == timeslot1 and room_idx == room1:
            return (course_id, professor, section, section_strength)
    return None

def generate_rooms():
    rooms = []
    for i in range(5):
        rooms.append((i, 60))
        rooms.append((i + 5, 120))
    return rooms

def generate_courses():
    courses = []
    for i in range(NUM_THEORY_COURSES):
        courses.append((i, "Theory"))
    for i in range(NUM_LAB_COURSES):
        courses.append((i + NUM_THEORY_COURSES, "Lab"))
    return courses

def generate_professors():
    professors = []
    for i in range(NUM_PROFESSORS):
        professors.append(i)
    return professors

rooms = generate_rooms()
courses = generate_courses()
sections = generate_sections()
professors = generate_professors()
timeslots = create_timeslots()

best_chromosome = genetic_algorithm(rooms, courses, sections, professors)
write_to_excel(best_chromosome, rooms, timeslots)

print("Timetable has been generated and saved to timetable-finals.xlsx.")


Gene 1: (1, 'Theory', 'A', 42, 0, 3, 1, 0)
Gene 2: (1, 'Theory', 'A', 42, 0, 1, 1, 0)
Gene 3: (2, 'Theory', 'A', 42, 1, 4, 1, 0)
Gene 4: (2, 'Theory', 'A', 42, 1, 1, 2, 0)
Gene 5: (3, 'Theory', 'A', 42, 2, 2, 2, 0)
Gene 6: (3, 'Theory', 'A', 42, 2, 0, 2, 0)
Gene 7: (4, 'Theory', 'A', 42, 3, 4, 0, 0)
Gene 8: (4, 'Theory', 'A', 42, 3, 2, 0, 4)
Gene 9: (5, 'Theory', 'A', 42, 4, 0, 0, 0)
Gene 10: (5, 'Theory', 'A', 42, 4, 2, 1, 0)
Gene 11: (26, 'Lab', 'A', 42, 5, 2, 4, 0)
Gene 12: (26, 'Lab', 'A', 42, 5, 4, 5, 3)
Gene 13: (1, 'Theory', 'B', 86, 6, 0, 1, 7)
Gene 14: (1, 'Theory', 'B', 86, 6, 2, 1, 7)
Gene 15: (2, 'Theory', 'B', 86, 7, 3, 0, 7)
Gene 16: (2, 'Theory', 'B', 86, 7, 1, 0, 7)
Gene 17: (3, 'Theory', 'B', 86, 8, 0, 2, 7)
Gene 18: (3, 'Theory', 'B', 86, 8, 2, 2, 7)
Gene 19: (4, 'Theory', 'B', 86, 9, 4, 0, 7)
Gene 20: (4, 'Theory', 'B', 86, 9, 1, 1, 7)
Gene 21: (5, 'Theory', 'B', 86, 0, 3, 2, 7)
Gene 22: (5, 'Theory', 'B', 86, 0, 1, 2, 7)
Gene 23: (26, 'Lab', 'B', 86, 1, 4, 4, 7)
Gen